# Squat Common

This files import libraries and contains common functions needed by the squat counter

In [1]:
# ===import the libraries===
import tensorflow as tf
# import tensorflow_hub as hub 
import numpy as np

import matplotlib.pyplot as plt
import matplotlib

import cv2 

print("Libraries imported.")

2024-05-20 12:30:31.748930: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 12:30:31.929732: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-20 12:30:31.935785: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-20 12:30:31.935801: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart d

Libraries imported.


In [2]:
# Download the model 
!wget -nc -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite

# Define model input size
input_size = 192

# Initialize the TFLite interpreter
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

In [4]:

# # ===Download an image for pose detection (if no "input_image.jpeg" file in directory)===
# import os 
# # Download the demo image, if doesn't exist in current folder
# if "input_image.jpeg" not in os.listdir():
#     os.system('curl -o input_image.jpeg https://images.pexels.com/photos/4384679/pexels-photo-4384679.jpeg --silent')
# input_path = "input_image.jpeg"

# print("Image loaded.")




KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}



# ===Define dict KEYPOINT_EDGE_INDS_TO_COLOR===
# ===Define _keypoints_and_edges_for_display===
KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def _keypoints_and_edges_for_display(keypoints_with_scores,
                                     height,
                                     width,
                                     keypoint_threshold=0.11):
    """Returns high confidence keypoints and edges for visualization.

    Args:
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    height: height of the image in pixels.
    width: width of the image in pixels.
    keypoint_threshold: minimum confidence score for a keypoint to be
      visualized.
    
    Returns:
    A (keypoints_xy, edges_xy, edge_colors) containing:
      * the absolute coordinates of all keypoints of all detected entities;
      * the absolute coordinates of all skeleton edges of all detected entities;
      * the colors in which the edges should be plotted.
    """
    keypoints_all = []
    keypoint_edges_all = []
    edge_colors = []
    _, num_instances, _, _ = keypoints_with_scores.shape
    
    for idx in range(num_instances):
        kpts_x = keypoints_with_scores[0, idx, :, 1]        # x coordinate, normalized
        kpts_y = keypoints_with_scores[0, idx, :, 0]        # y coordinate, normalized
        kpts_scores = keypoints_with_scores[0, idx, : , 2]  # score
        kpts_absolute_xy = np.stack((width*kpts_x, height*kpts_y), axis=1)
        kpts_above_thresh_absolute_xy = kpts_absolute_xy[
            kpts_scores > keypoint_threshold, :]
        keypoints_all.append(kpts_above_thresh_absolute_xy)

        for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
            kpt_idx1 , kpt_idx2 = edge_pair[0], edge_pair[1]
            if (kpts_scores[kpt_idx1] > keypoint_threshold and
               kpts_scores[kpt_idx2] > keypoint_threshold):
                x_start, y_start = kpts_absolute_xy[kpt_idx1]
                x_end, y_end = kpts_absolute_xy[kpt_idx2]
                line_seg = np.array([[x_start, y_start], [x_end, y_end]])
                keypoint_edges_all.append(line_seg)
                edge_colors.append(color)
    if keypoints_all:
        keypoints_xy = np.concatenate(keypoints_all)
    else:
        keypoints_xy = np.zeros((0, 17, 2))
    
    if keypoint_edges_all:
        edges_xy = np.stack(keypoint_edges_all, axis=0)
    else:
        edges_xy = np.zeros((0, 2, 2))
    return keypoints_xy, edges_xy, edge_colors


                                         
# ===Define: draw_prediction_on_image===
# changes: we will need to show too many pictures,
#      so matplotlib is configured to "Agg" mode, and forced GC is used
from matplotlib.collections import LineCollection
from matplotlib import patches

import gc

def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):

    """Draws the keypoint predictions on image.

    Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.
    with_padding: if the result keeps edge padding. Normally if the input image
      to the MoveNet has padding but in display won't padding, with_padding = 0
    
    Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
    """
        
    matplotlib.use("Agg")     # important, memery leak will orrur if without this statement
    
    height, width, channel = image.shape
    aspect_ratio = width / height

    fig, ax = plt.subplots( figsize=( 12*aspect_ratio, 12 ) )
    fig.tight_layout(pad = 0)
    ax.margins(0)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    plt.axis("off")

    ax.imshow(image)
    line_segments = LineCollection([], linewidths=(4), linestyle = 'solid' )
    ax.add_collection(line_segments)

    scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

    keypoint_locs, keypoint_edges, edge_colors = ( _keypoints_and_edges_for_display(
        keypoints_with_scores, height, width) )
        
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
    scat.set_offsets(keypoint_locs)

    if crop_region is not None:
        xmin = max(crop_region['x_min'] * width, 0.0)
        ymin = max(crop_region['y_min'] * height, 0.0)
        rec_width = min(crop_region['x_max'], 0.99) * width - xmin
        rec_height = min(crop_region['y_max'], 0.99) * height - ymin
        rect = patches.Rectangle(
            (xmin,ymin),rec_width,rec_height,
            linewidth=1,edgecolor='b',facecolor='none')
        ax.add_patch(rect)

    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close("all")
    plt.clf()
    gc.collect()     # important, memery leak will orrur if without this statement
    return image_from_plot


        
# ===Defined IPython widget: stopButton===
from IPython.display import display, Image
import IPython
import ipywidgets as widgets
import threading

stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)


print('''
The following functions have been imported:
   movenet(input_image)
   _keypoints_and_edges_for_display(keypoints_with_scores, height, width, keypoint_threshold=0.11)
   draw_prediction_on_image(image, keypoints_with_scores, crop_region=None, close_figure=False, output_image_height=None)
The following global variables have been assigned:
   module, model_name, input_size, KEYPOINT_DICT, KEYPOINT_EDGE_INDS_TO_COLOR
=======================================
'''
)

if __name__ != "__main__":
    print("====== Importing from step 4 ... ======")



The following functions have been imported:
   movenet(input_image)
   _keypoints_and_edges_for_display(keypoints_with_scores, height, width, keypoint_threshold=0.11)
   draw_prediction_on_image(image, keypoints_with_scores, crop_region=None, close_figure=False, output_image_height=None)
The following global variables have been assigned:
   module, model_name, input_size, KEYPOINT_DICT, KEYPOINT_EDGE_INDS_TO_COLOR



In [5]:
def load_image_to_tf(image):
    # convert image to tf format
    # input argument "image" can be Array of image, or a string of the path to an image in jpeg format
    if isinstance(image, str):
        image = tf.io.read_file(image)
        image_converted = tf.io.decode_jpeg(image)
    else: 
        image_converted = tf.convert_to_tensor(image)
    return image_converted


In [6]:
def tf_image_to_model(image, width, height):
    # convert to model required format
    image_expand_dim = tf.expand_dims(image, 0)
    input_image = tf.image.resize_with_pad(image_expand_dim, width, height)
    return input_image



In [7]:
def show_image(image):
    print(image.shape)
    # show an image using matplotlib. 
    # matplotlib is changed to "Agg" mode in the end to disable interaction to avoid memory leak
    matplotlib.use('module://ipykernel.pylab.backend_inline')
    plt.clf()
    if isinstance(image, np.ndarray):
        plt.imshow(image / 255)
    else:
        plt.imshow(image.numpy() / 255.0)
    plt.show()
    matplotlib.use('Agg')


In [8]:
def get_keypoints_with_scores_from_image_with_movenet(image, width, height):
    """
    Return the keypoints with scores from a given image using Movenet

    Args:
    image: can be Array of image, or a string of the path to the image
    width: width of input image Model required
    height: height of input image Model required

    Return: 
    A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    """
    image_converted = load_image_to_tf(image)
    input_image = tf_image_to_model(image_converted, width, height)
    keypoints_with_scores = movenet(input_image)
    return keypoints_with_scores


In [9]:
# Display function
# ================
def view( button, source=0, rotate=0, process=(lambda x:x) ):
    """
    Args:
    button: An IPywidget button to control the function. The display stops if button.value == True.
    Source: An optional integer or filename to specify the source of the video stream.
    Rotate: optional integer. Set to 0 by default (not rotated). Set to 1 will rotate by 90 deg.
    process: optional function that processes each frame through process(frame) before displaying.
        Set to identity function by default.
    """
    display_handle=display("Please wait...", display_id=True)
    
    cap = cv2.VideoCapture(source)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

    # print("something")
    
    while True:
        ret, frame = cap.read()
        if button.value==True or not ret:
            cap.release()
            display_handle.update(None)
            button.value = False
            break
        # frame = cv2.flip(frame, 1) # if your camera reverses your image

        # Rotate the frame if rotate==1
        if rotate:
            # frame = cv2.transpose(frame)
            frame = cv2.flip(frame, 0)
        
        frame = process(frame)
        
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))